# Installing libraries

In [1]:
!pip install openai
!pip install llama-index
!pip install trulens_eval
!pip install pymilvus
!pip install llama-index-vector-stores-milvus

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 75.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 76.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 78.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 83.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 71.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

# Coding

In [24]:
from openai import OpenAI
import os
os.environ["OPENAI_API_KEY"] = "sk-..."

Standard LlamaIndex query engine. Using VectorStoreIndex

In [71]:
from llama_index.core import VectorStoreIndex,SimpleDirectoryReader
documents=SimpleDirectoryReader("preprocessed_data").load_data()
# print("Document ID:", documents[0].doc_id) seeing what the doc looks like
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine(similarity_top_k=4)

Retrievers and postprocessors

In [72]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor

retriever=VectorIndexRetriever(index=index,similarity_top_k=4)
postprocessor=SimilarityPostprocessor(similarity_cutoff=0.8)

query_engine=RetrieverQueryEngine(retriever=retriever,
                                  node_postprocessors=[postprocessor])

Using Milvus as the vector store

In [ ]:
from pymilvus import connections, Collection
MILVUS_HOST = 'localhost'  # the location on my laptop in which milvus is running
MILVUS_PORT = '2379'
connections.connect(host=MILVUS_HOST, port=MILVUS_PORT)

In [ ]:
#Setting up the vecor store.
from llama_index.vector_stores.milvus import MilvusVectorStore
from llama_index.core.storage.storage_context import StorageContext
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

vector_store = MilvusVectorStore(dim=1536, host="localhost", port=19530, overwrite=True)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)

# Retriever and Postproccesor query engine as this produced best results in early testing
retriever=VectorIndexRetriever(index=index,similarity_top_k=4)
postprocessor=SimilarityPostprocessor(similarity_cutoff=0.8)

query_engine=RetrieverQueryEngine(retriever=retriever,
                                  node_postprocessors=[postprocessor])

Running questions - this is just for when I was testing

In [53]:
from llama_index.core.response.pprint_utils import pprint_response
response = query_engine.query("What happens to my body if I do not get enough sleep?")
pprint_response(response,show_source=True)

Insufficient sleep can lead to various negative effects on the body, such as reduced blood flow to the skin in the face, resulting in a dull complexion with dark areas. It can also cause dark circles under the eyes, more wrinkles due to decreased collagen production, puffy eyes, and a lack of a healthy glow. Additionally, insufficient sleep can impact overall appearance, making individuals appear less happy.


Import Tru Lens

In [54]:
from trulens_eval import Tru
tru = Tru()

Creating the feedback functions that will be used in evaluation of the RAG LLM

In [55]:
import numpy as np

# Initialize provider class
from trulens_eval.feedback.provider.openai import OpenAI
openai = OpenAI()

# select context to be used in feedback. the location of context is app specific.
from trulens_eval.app import App
context = App.select_context(query_engine)

# imports for feedback
from trulens_eval import Feedback

# Define a groundedness feedback function
from trulens_eval.feedback import Groundedness
grounded = Groundedness(groundedness_provider=OpenAI())
f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons)
    .on(context.collect()) # collect context chunks into a list
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator)
)

# Question/answer relevance between overall question and answer.
f_qa_relevance = Feedback(openai.relevance).on_input_output()

# Question/statement relevance between question and each context chunk.
f_qs_relevance = (
    Feedback(openai.qs_relevance)
    .on_input()
    .on(context)
    .aggregate(np.mean)
)

✅ In groundedness_measure_with_cot_reasons, input source will be set to __record__.app.query.rets.source_nodes[:].node.text.collect() .
✅ In groundedness_measure_with_cot_reasons, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In qs_relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In qs_relevance, input statement will be set to __record__.app.query.rets.source_nodes[:].node.text .


To asses the model we wrap the query engine in a TruLLama instance to asses the model, at this point we also name the app to identify it later. We also provide the feedback functions created above.

In [90]:
from trulens_eval import TruLlama
tru_query_engine_recorder = TruLlama(query_engine,
    app_id='LlamaIndex_App8',
    feedbacks=[f_groundedness, f_qa_relevance, f_qs_relevance])

Retrieve records and feedback

In [100]:
with tru_query_engine_recorder as recording:
    query_engine.query("Can you list ways that heart rate can be measured?")

In [101]:
# The record of the app invocation can be retrieved from the `recording`:
rec = recording.get()

display(rec)

Record(record_id='record_hash_fbe810723cfff883cbd72a36efbbfd02', app_id='LlamaIndex_App8', cost=Cost(n_requests=2, n_successful_requests=2, n_classes=0, n_tokens=1131, n_stream_chunks=0, n_prompt_tokens=1106, n_completion_tokens=25, cost=0.0016925), perf=Perf(start_time=datetime.datetime(2024, 3, 4, 16, 59, 34, 566721), end_time=datetime.datetime(2024, 3, 4, 16, 59, 37, 610590)), ts=datetime.datetime(2024, 3, 4, 16, 59, 37, 611851), tags='-', meta=None, main_input='Can you list ways that heart rate can be measured?', main_output='Heart rate can be measured by listening to the chest and sensing minute changes in blood flow via photoplethysmography.', main_error=None, calls=[RecordAppCall(stack=[RecordAppCallMethod(path=Lens().app, method=Method(obj=Obj(cls=llama_index.core.query_engine.retriever_query_engine.RetrieverQueryEngine, id=140331591791424, init_bindings=None), name='query')), RecordAppCallMethod(path=Lens().app, method=Method(obj=Obj(cls=llama_index.core.query_engine.retriever

In [102]:
# The results of the feedback functions can be retrieved using the `wait_for_feedback_result` method. Also prints a score for your metrics below seperate to the average one printed using get_leaderboard()
for feedback, feedback_result in rec.wait_for_feedback_results().items():
    print(feedback.name, feedback_result.result)


groundedness_measure_with_cot_reasons 1.0
relevance 0.8
qs_relevance 0.2


In [103]:
records, feedback = tru.get_records_and_feedback(app_ids=["LlamaIndex_App8"])

records.head()

tru.get_leaderboard(app_ids=["LlamaIndex_App8"])

,qs_relevance,groundedness_measure_with_cot_reasons,relevance,latency,total_cost
app_id,,,,,
LlamaIndex_App8,0.2,1.0,0.933333,3.666667,0.00164


# Data Preprocessing

In [33]:
# Extracting the contents from the pdf slides.
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    extracted_text = ""
    pdf_document = fitz.open(pdf_path)

    for page_number in range(pdf_document.page_count):
        current_page = pdf_document[page_number]
        extracted_text += current_page.get_text()

    pdf_document.close()
    return extracted_text

pdf_file_path = 'data/HumanSensing.pdf'
extracted_text = extract_text_from_pdf(pdf_file_path)
extracted_text

"- 1 -\nMOOC 2: Sensing the Human\n- 2 -\nOverview\n• In MOOC-1 we looked at the human brain and \nhuman memory and how it works, and often \ndoesn’t, and how we use search and \ninformation seeking to support our memory \nfunction.  \n• In MOOC-2 we’ll look at technologies for \nsensing the human body and where it is and \nwhat its doing, and applications which use \nsuch sensing\n• This covers sensing location, sensing the \nbody’s activities, sensing our physiology with a \nparticular focus on sleep applications, and \nfinishing with sensing activity in the brain\n- 3 -\nWhy Know About User Context ?\n•\nThere are many reasons for wanting to sense the human \nbody and where it is and what its doing … health is an \nobvious one\n•\nThere’s also information-finding, when we search for \ninformation it is usually about documents and pages, and \nuser query & matching but knowing what we’re doing \nand/or where we are when we’re seeking information will \nhelp improve the systems we use

Creating the function to process the text and split on page number. 

In [70]:
import re

def split_text_into_pages(text):
    # Replace bullet points with a full stop
    text = text.replace('•', '.')
    text = text.lower()

    # Trying to define the pattern for page numbers
    page_pattern = re.compile(r'\b\d+\b')

    # Find all occurrences of the pattern
    matches = page_pattern.finditer(text)

    page_numbers = []
    page_contents = []

    # Iterate through the matches
    matches_iter = iter(matches)
    for match in matches_iter:
        # Extract page number and content
        page_number = int(match.group().strip())

        # End position of the current match
        end_pos = match.end()
        # Position of next match
        next_match = next(matches_iter, None)
        content = text[end_pos:next_match.start()] if next_match else text[end_pos:]

        page_numbers.append(page_number)
        page_contents.append(content.strip())

    return page_numbers, page_contents

page_numbers, page_contents = split_text_into_pages(extracted_text)

for page_number, content in zip(page_numbers, page_contents):
    print(f"Page Number: {page_number}\nContent: {content}\n")


Page Number: 1
Content: -
mooc

Page Number: 2
Content: -
overview
. in mooc-

Page Number: 2
Content: we’ll look at technologies for 
sensing the human body and where it is and 
what its doing, and applications which use 
such sensing
. this covers sensing location, sensing the 
body’s activities, sensing our physiology with a 
particular focus on sleep applications, and 
finishing with sensing activity in the brain
-

Page Number: 4
Content: -
kinds of context
. there is no universally agreed 
classification of “user context”, just an 
arbitrary one, different kinds of user 
context that can be captured.
. i divide it into

Page Number: 5
Content: -
week

Page Number: 1
Content: : sensing actual location
. gps how it works, in smartphones, 
computers, watches, cars, etc.
. a review of some location based 
services (lbs) that use actual location 
. strava and mapmyrun for the individual
. google and apple tracking … iphone 
locations … to illustrate ethical and data 
privacy issues.
-

Here I am converting the lists produced by the above function to create a dataframe and export it to a csv file in a folder I created called preproccessed_data.

In [47]:
import pandas as pd
data = data = {'Page': page_numbers,
        'Contents': page_contents}
df = pd.DataFrame(data)
print(df)

# Created approximately 14 extra splits through the text
len(df)

     Page                                           Contents
0       1                                            -\nMOOC
1       2                            -\nOverview\n. In MOOC-
2       2  we’ll look at technologies for \nsensing the h...
3       4  -\nKinds of Context\n. There is no universally...
4       5                                            -\nWeek
..    ...                                                ...
225   207                                P300 elicitation\n-
226   209                                P300 elicitation\n-
227   211                                P300 elicitation\n-
228   213                                P300 elicitation\n-
229   215  P300 wave\n.\nThere’s nothing special about th...

[230 rows x 2 columns]


230

In [48]:
csv_file_path = "preprocessed_data/preproccesed.csv"
df.to_csv(csv_file_path, index=False)